In [1]:
# Taken from https://github.com/alexjungaalto/ResearchPublic/blob/master/RoadMarkingHelsinki/RoadMarkingMonitoring.ipynb

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import datetime
import numpy as np
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from owslib.wms import WebMapService
from skimage.feature import hog
import io

wms = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms', version='1.1.1')


# define the bounding box of the entire city area 

xmin = 24.92
ymin = 60.15
xmax = 24.99
ymax = 60.20

# we divide the entire city area evenly into nr_patches_x x nr_patches_y square patches 

nr_patches_x = 50 
nr_patches_y = 50 

patch_x = (xmax-xmin)/nr_patches_x  # width of an individual patch
patch_y = (ymax-ymin)/nr_patches_y  # height of a single patch

In [ ]:
# choose one particular patch 

nr_x = 11 
nr_y = 10

# determine the bounding box for this particular patch 

patch_box = (xmin+nr_x*patch_x,ymin+nr_y*patch_y,xmin+(nr_x+1)*patch_x,ymin+(nr_y+1)*patch_y) 

ortholayer = 'avoindata:Ortoilmakuva_2019_5cm' # choose layer with aerial photographs 
cs = 'EPSG:4326'
resx=500    # choose number of pixels in one direction 
res = (resx,resx)

fig, axes = plt.subplots(1, 2, figsize=(15, 15))

img = wms.getmap(layers=[ortholayer],srs=cs,bbox=patch_box,size=res,format='image/jpeg',transparent=True)
A_1 = np.asarray(Image.open(io.BytesIO(img.read())))
axes[0].imshow(A_1)
axes[0].set_title("Layer obtained from the aerial photographs.") 


img = wms.getmap(layers=['Karttasarja'],srs=cs,bbox=patch_box,size=res,format='image/jpeg',transparent=True)
A_1_layer_1 = np.asarray(Image.open(io.BytesIO(img.read())))
axes[1].imshow(A_1_layer_1)
axes[1].set_title("Layer obtained from the building map.") 

plt.show()

# Getting the data

In [ ]:
import geopandas
import fiona
import pandas as pd
import numpy as np
from owslib.wms import WebMapService
import os

file=geopandas.read_file("klinj_suojatie.gpkg")
df=pd.DataFrame(file)
df.geometry=df.geometry.astype("str")
df["geometry"]=df["geometry"].str.replace("LINESTRING ","").str.replace(",","").str.replace("(","").str.replace(")","")

df[["min_x","max_y","max_x","min_y","a","b"]]=df["geometry"].str.split(" ",expand=True)
df=df.drop(columns=["a","b","geometry"])
df["id"]=df["gml_id"].str.replace("klinj_suojatie.","")

df["min_x"]=df["min_x"].astype(float)
df["min_y"]=df["min_y"].astype(float)
df["max_x"]=df["max_x"].astype(float)
df["max_y"]=df["max_y"].astype(float)

# Fixing inconsistencies 
df.min_x, df.max_x = np.where(df.min_x > df.max_x, [df.max_x, df.min_x], [df.min_x, df.max_x])
df.min_y, df.max_y = np.where(df.min_y > df.max_y, [df.max_y, df.min_y], [df.min_y, df.max_y])


resolution = 100 


# choose layers which are used for the pedestrian crossing images 

layer = "Ortoilmakuva_2019_5cm"

#layer = "Vaaravariortoilmakuva_2019_20cm"

folder_name ="PedestrianCrossingImg/resolution_"+str(resolution)+"_layer_"+layer+"/" 


if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
filelist=os.listdir(folder_name)
nrfiles = len(filelist)

if nrfiles < 100 : 

    wms = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms', version='1.3.0')

    for i in range(0,len(df)):
        # determine bounding box of ith pedestrian crossing 
        minx = df.loc[i,"min_x"] 
        miny = df.loc[i,"min_y"]
        maxx = df.loc[i,"max_x"]
        maxy = df.loc[i,"max_y"]
        deltax = (maxx-minx)
        deltay = (maxy-miny)
        roi = (minx-deltax,miny-deltay,maxx+deltax,maxy+deltay)
    
        #read in image patch from server
 
        img=wms.getmap(layers=[layer],srs="EPSG:3879",bbox=roi,size=(resolution,resolution),format='image/png',transparent=True)
        filename =folder_name + str(df.loc[i,"id"])+".png"
        #filename  = "pedestrian_crossing" + str(i)+".png"
        out = open(filename, 'wb')
        out.write(img.read())
        out.close()